# Домашнее задание Аэропорты

In [1]:
#подгружаем необходимые библиотеки
import pandas as pd
import numpy as np

In [2]:
#считываем датасет с аэропортами
airports=pd.read_csv('airports.csv',header=None,index_col='Airport ID',names=['Airport ID','Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone','DST','Tz','Type','Source'])

#список стран Европы
europe=np.array(['Belgium', 'Germany', 'Estonia', 'Finland', 'United Kingdom',
'Guernsey', 'Jersey', 'Isle of Man', 'Netherlands', 'Ireland',
'Denmark', 'Luxembourg', 'Norway', 'Poland', 'Sweden', 'Spain',
'Albania', 'Bulgaria', 'Cyprus', 'Croatia', 'France', 'Greece',
'Hungary', 'Italy', 'Slovenia', 'Czech Republic', 'Malta',
'Austria', 'Portugal', 'Bosnia and Herzegovina', 'Romania',
'Switzerland', 'Turkey', 'Moldova', 'Macedonia', 'Gibraltar',
'Serbia', 'Montenegro', 'Slovakia', 'Ukraine', 'Russia', 'Belarus',
'Latvia', 'Lithuania','Faroe Islands', 'Greenland', 'Iceland'])

#из исходного датасета отбираем строки с Европой
europe_df=airports[airports['Country'].isin(europe)]

#отобрали из него только те страны, где трехзначный код IATA
europe_df=europe_df[europe_df['IATA'].str.len()==3]

#Заменяем пропуски
europe_df=europe_df.replace('\\N',np.nan)

#Теперь работаем с датасетом с маршрутами (файл "routes.csv")
routes=pd.read_csv('routes.csv',index_col=0,header=None,
                   names=['Airline','Airline ID','Source airport','Source airport ID','Destination Airport','Destination Airport ID','Codeshare','Stops','Equipment'])

#Заменяем пропуски
routes=routes.replace('\\N',np.nan)

#отбираем маршруты, где фигурируют аэропорты Европы
routes_df=routes[routes['Source airport'].isin(europe_df['IATA'])|
                 routes['Destination Airport'].isin(europe_df['IATA'])]


#Посчитали взлеты и посадки
source=routes_df.groupby('Source airport')["Destination Airport"].count()
desten=routes_df.groupby('Destination Airport')["Source airport"].count()

# ВНИМАНИЕ!!! тут налажали многие!!! здесь корректное  объединение, иначе 1+NaN=NaN
# Важно fill_value=0
# например аэрапорт ADE=1, ZYL=1, а  они пропущены у многих
fly=source.add(desten,fill_value=0).astype(int)

#Слили результат 
res=europe_df.merge(fly.to_frame(name='Fly'), left_on='IATA', right_index=True).set_index('Name')
res.head(5)

,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz,Type,Source,Fly
Name,,,,,,,,,,,,,
Narsarsuaq Airport,Narssarssuaq,Greenland,UAK,BGBW,61.160500,-45.425999,112,-3,E,America/Godthab,airport,OurAirports,10
Godthaab / Nuuk Airport,Godthaab,Greenland,GOH,BGGH,64.190903,-51.678101,283,-3,E,America/Godthab,airport,OurAirports,18
Kangerlussuaq Airport,Sondrestrom,Greenland,SFJ,BGSF,67.012222,-50.711603,165,-3,E,America/Godthab,airport,OurAirports,16
Thule Air Base,Thule,Greenland,THU,BGTL,76.531197,-68.703201,251,-4,E,America/Thule,airport,OurAirports,4
Akureyri Airport,Akureyri,Iceland,AEY,BIAR,65.660004,-18.072701,6,0,N,Atlantic/Reykjavik,airport,OurAirports,2


In [3]:
res.groupby('Country')['Fly'].nlargest(3).to_frame(name='Fly')

Fly
Country                Name                                            
Albania                Tirana International Airport Mother Teresa    72
Austria                Vienna International Airport                 616
                       Salzburg Airport                              69
                       Graz Airport                                  38
Belarus                Minsk National Airport                       108
Belgium                Brussels Airport                             631
                       Brussels South Charleroi Airport             168
                       Liège Airport                                 17
Bosnia and Herzegovina Sarajevo International Airport                32
                       Banja Luka International Airport               6
                       Mostar International Airport                   2
Bulgaria               Sofia Airport                                132
                       Burgas Airport                                24
                       Varna Airport                                 18
Croatia                Dubrovnik Airport                            121
                       Split Airport                                 97
                       Zagreb Airport                                85
Cyprus                 Larnaca International Airport                190
                       Paphos International Airport                  85
                       Ercan International Airport                   28
Czech Republic         Václav Havel Airport Prague                  361
                       Karlovy Vary International Airport            10
                       Ostrava Leos Janáček Airport                   8
Denmark                Copenhagen Kastrup Airport                   457
                       Billund Airport                               94
                       Aalborg Airport                               41
Estonia                Lennart Meri Tallinn Airport                  76
                       Tartu Airport                                  4
                       Kuressaare Airport                             2
Faroe Islands          Vagar Airport                                  8
...                                                                 ...
Portugal               Francisco de Sá Carneiro Airport             202
Romania                Henri Coandă International Airport           227
                       Cluj-Napoca International Airport             42
                       Timişoara Traian Vuia Airport                 26
Russia                 Domodedovo International Airport             649
                       Sheremetyevo International Airport           397
                       Pulkovo Airport                              368
Serbia                 Belgrade Nikola Tesla Airport                171
                       Priština International Airport                42
Slovakia               M. R. Štefánik Airport                        40
                       Košice Airport                                11
Slovenia               Ljubljana Jože Pučnik Airport                 66
Spain                  Barcelona International Airport              783
                       Adolfo Suárez Madrid–Barajas Airport         661
                       Palma De Mallorca Airport                    548
Sweden                 Stockholm-Arlanda Airport                    439
                       Gothenburg-Landvetter Airport                106
                       Stockholm Skavsta Airport                     62
Switzerland            Zürich Airport                               494
                       Geneva Cointrin International Airport        329
                       Bern Belp Airport                             24
Turkey                 Atatürk International Airport                719
                       Sabiha Gökçen International Airport          290
                       Antalya Inter